In [ ]:
%matplotlib inline
import matplotlib
import os              
import pandas as pd    
import seaborn as sns
import math
import numpy as np

In [ ]:
# Read in the bridge and roads
df1 = pd.read_csv("https://github.com/yasinsariboun/assignment2/raw/main/_roads3.csv")
df3 = pd.read_excel("https://github.com/yasinsariboun/assignment2/raw/main/BMMS_overview.xlsx")

In [ ]:
def road_function(df,df2, mainroad1,mainroad2):
    mainroads = df.loc[(df['road']==mainroad1)|(df['road']==mainroad2)] #select the two mainroads
    #select all rows that can indicate crossroads of 25 km or longer
    mainroads2=mainroads.loc[(mainroads['type']== 'CrossRoad') |(mainroads['type']== 'SideRoad,Right') | (mainroads['type']=='SideRoad,Left') | (mainroads['type']=='CrossRoad / KmPost') | (mainroads['type']=='SideRoad,Left / SideRoad,Right') | (mainroads['type']=='SideRoad,Left / KmPost') | (mainroads['type']=='SideRoad,Right / SideRoad,Right')]
    mainroads2=mainroads2.reset_index() 
    crossings=mainroads2['name'].str.extract('.*([ZNR](\d{4}|\d{3}|\d{1})).*') #extract the names of the crossroads present in the column 'name'
    for i in range(len(mainroads2)): #add three columns in which the lattitude, longitude and mainroad are defined. Now we have a dataframe with all relevant crossings, where they are and between which roads they take place.
        crossings.loc[i,1] = mainroads2.iloc[i,4] 
        crossings.loc[i,2] = mainroads2.iloc[i,5]
        crossings.loc[i,3] = mainroads2.iloc[i,1]
    allotherroads=df.drop(mainroads.index)  #now we are goinig to select the roads from the original dataframe that are mentioned in 'crossings' as the sideroads
    sideroads2=pd.DataFrame() 
    for j in range(len(crossings)):
        b=allotherroads.loc[allotherroads['road']==crossings[0][j]]
        if b['chainage'].max()>25.0:  #and if the sideroad is 25 km or longer, we add it to a dataframe with all other sideroads
            sideroads2=pd.concat([sideroads2,b]).drop_duplicates().reset_index(drop=True)
    dbroads=pd.concat([mainroads,sideroads2]) #which we then combine with the mainroads to form one big dataframe with the mainroads and the sideroads
    crossings = crossings[crossings[0].isin(dbroads['road'].unique())] #now we update the crossings dataframe to contain only the crossings of N1 and N2 with roads longer than 25 km
    crossings.columns=['crossroad','lat','long','mainroad'] #we rename the columns of 'crossings'
    crossings=crossings[['mainroad','crossroad','lat','long']]  #we change the order of the columns of 'crossings'
    dbroads.loc[:,'model_type']='link'  #now we add a new column 'model_type' that contains 'link' for all rows
    #and we remove all chainage points that are not relevant for us. We only look at the kmposts and the crossings.
    dbroadsclean = dbroads.loc[(dbroads['type']== 'Others') | (dbroads['type']== 'KmPost')|(dbroads['type']== 'CrossRoad') |(dbroads['type']== 'SideRoad,Right') | (dbroads['type']=='SideRoad,Left') | (dbroads['type']=='CrossRoad / KmPost') | (dbroads['type']=='SideRoad,Left / SideRoad,Right') | (dbroads['type']=='SideRoad,Left / KmPost') | (dbroads['type']=='SideRoad,Right / SideRoad,Right')]
    dbroadsclean2=dbroadsclean.copy()
    for n in range(len(dbroadsclean2)): #all beginnings and ends of roads are defined as 'sourcesink' in column 'model_type'
        if n==0:
            dbroadsclean2.iloc[n,8]='sourcesink'
            continue
        if n==(len(dbroadsclean2)-1):
            dbroadsclean2.iloc[n,8]='sourcesink'
            continue
        if dbroadsclean2.iloc[n,0]!=dbroadsclean2.iloc[n+1,0]:
            dbroadsclean2.iloc[n,8]='sourcesink'
            dbroadsclean2.iloc[n+1,8]='sourcesink'
    dfd = dbroadsclean2.copy()
    for m in range(len(crossings)): #all points from the 'dbroadsclean2' dataset arecompared with the crossings file. All rows that define a chainage point at an intersection are defined as 'intersection' in column 'model_type'
        dfd.loc[(dfd['road']==crossings.iloc[m,0]) &(dfd['lat']==crossings.iloc[m,2]) & (dfd['lon']==crossings.iloc[m,3]),'model_type']='intersection'
        dfd.loc[(dfd['road']==crossings.iloc[m,1]) &(dfd['lat']==crossings.iloc[m,2]) & (dfd['lon']==crossings.iloc[m,3]),'model_type']='intersection'
    return dfd , crossings

In [ ]:
q = road_function(df1, df3, 'N1', 'N2')

In [ ]:
q[0].to_csv('stap1.csv')

In [ ]:
q[1]

,mainroad,crossroad,lat,long
3,N1,N2,23.7061,90.521527
4,N1,N105,23.6904,90.546583
9,N1,Z1402,23.5307,90.781610
11,N1,Z1044,23.5307,90.792444
12,N1,Z1042,23.5127,90.871416
17,N1,N102,23.479,91.118166
23,N1,Z1048,23.1497,91.325499
24,N1,Z1031,23.0615,91.361667
25,N1,Z1031,23.0479,91.365139
27,N1,N104,23.0096,91.381360


In [ ]:
def bridge_function(df2,mainroad1,mainroad2,crossings):
    bridgesonmainroads= df2.loc[(df2['road']==mainroad1) | (df2['road']==mainroad2)]
    allotherbridges=df2.drop(bridgesonmainroads.index)  #now we are goinig to select the roads from the original dataframe that are mentioned in 'crossings' as the sideroads
    bridgesonsideroads=pd.DataFrame() 
    for j in range(len(crossings)):
        b=allotherbridges.loc[allotherbridges['road']==crossings.iloc[j,1]]
        bridgesonsideroads=pd.concat([bridgesonsideroads,b])
    bridges1=pd.concat([bridgesonmainroads,bridgesonsideroads]) #which we then combine with the mainroads to form one big dataframe with the mainroads and the sideroads
    # Delete the left (L) side bridges
    df4=bridges1[bridges1.name.str.endswith("(L)")]
    df4=pd.concat([bridges1, df4, df4]).drop_duplicates(keep=False)
    # Delete the left ( L ) side bridges
    df5=df4[df4.name.str.endswith("( L )")]
    df5=pd.concat([df4, df5, df5]).drop_duplicates(keep=False)
    # Delete the left (LEFT) side bridges
    df6=df5[df5.name.str.endswith("(LEFT)")]
    df6=pd.concat([df5, df6, df6]).drop_duplicates(keep=False)
    # Delete the left (Left) side bridges
    df7=df6[df6.name.str.endswith("(Left)")]
    df7=pd.concat([df6, df7, df7]).drop_duplicates(keep=False)
    # Drop those duplicates if they have the exact same number of KM meaning that they are on the exact some spot. 
    df8 = df7[~((df7[['road', 'km']].duplicated(keep=False)))]
    df9 = df8[df8['type'].isin(['PC Girder Bridge','RCC Girder Bridge','RCC Bridge','Box Culvert','Slab Culvert'])]
    df10=df9.copy()
    df10.loc[:,'model_type']='bridge'
    df10.to_csv('stap3.csv')
    return df10

In [ ]:
f2=bridge_function(df3,'N1','N2',q[1])

In [ ]:
f2

,road,km,type,LRPName,name,length,condition,structureNr,roadName,chainage,width,constructionYear,spans,zone,circle,division,sub-division,lat,lon,EstimatedLoc,model_type
0,N1,1.800,Box Culvert,LRP001a,.,11.3,A,117861,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,1.800,19.50,2005.0,2.0,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.698739,90.458861,interpolate,bridge
1,N1,4.925,Box Culvert,LRP004b,.,6.6,A,117862,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,4.925,35.40,2006.0,1.0,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.694664,90.487775,interpolate,bridge
3,N1,10.880,Box Culvert,LRP010b,NOYAPARA CULVERT,6.3,A,112531,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,10.880,12.20,1992.0,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,23.694391,90.537574,interpolate,bridge
4,N1,10.897,Box Culvert,LRP010c,ADUPUR CULVERT,6.3,A,112532,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,10.897,12.20,1984.0,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,23.694302,90.537707,interpolate,bridge
5,N1,11.296,Box Culvert,LRP011a,NAYABARI KASPUR BOX CULVERT,8.3,A,101110,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,11.296,21.45,1986.0,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,23.692360,90.540918,interpolate,bridge
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15317,N208,28.175,Box Culvert,LRP028e,LALA PUR BOX CULVERT,3.0,B,100225,Moulvibazar-Rajnagar-Fenchuganj-Sylhet Road,28.175,12.20,1994.0,1.0,Sylhet,Moulavi Bazar,Moulavi Bazar,Moulavi Bazar,24.637082,91.914663,interpolate,bridge
15318,N208,36.154,Box Culvert,LRP036e,MOLIKPUR BOX CULVERT,19.0,B,100257,Moulvibazar-Rajnagar-Fenchuganj-Sylhet Road,36.154,12.20,1985.0,3.0,Sylhet,Sylhet,Sylhet,Biswanath,24.702575,91.929049,interpolate,bridge
15327,N208,57.089,Box Culvert,LRP057b,KADAM TALI BOX CULVERT,5.6,B,100272,Moulvibazar-Rajnagar-Fenchuganj-Sylhet Road,57.089,9.70,1985.0,1.0,Sylhet,Sylhet,Sylhet,Biswanath,24.878083,91.875333,bcs1,bridge
16372,N208,23.646,RCC Girder Bridge,LRP024b,Kutalpur Bridge,10.3,C,100280,Moulvibazar-Rajnagar-Fenchuganj-Sylhet Road,23.646,4.00,1982.0,3.0,Sylhet,Sylhet,Sylhet,Biswanath,24.599935,91.899895,interpolate,bridge


In [ ]:
def merge_function(roads,bridges):
  
    roadsmerge = roads[['road','model_type','name','lat','lon', 'chainage']]
    bridgesmerge = bridges[['road','model_type','name','lat','lon', 'chainage','length','condition']] 
    merge = pd.concat([roadsmerge, bridgesmerge])
    merge.to_csv('stap2.csv')
    mergesorted = merge.sort_values(['road','chainage'])
    mergesorted.to_csv('stap3.csv')
    mergesorted = mergesorted.reset_index(drop=True)
    mergesorted.to_csv('stap4.csv')
    mergesorted['id'] = range(1000000,1000000+len(mergesorted))
    #based on chainage we are going to define the link length. We define the link length from a row as the chainage from the next point minus the chainage from its own point.
    #However since we are driving opposite from the chainage (the chainage goes down from Chittagong to Dhaka), we subtract the chainage from the current point from the chainage from the next point to arrive at the length of a link.

    for i in range(0, len(mergesorted.index)-1):
        if math.isnan(mergesorted.loc[i,'length']):
            if mergesorted.loc[i+1,'chainage'] > mergesorted.loc[i,'chainage']:
                mergesorted.loc[i,'length'] = 1000* abs(mergesorted.loc[i+1,'chainage'] - mergesorted.loc[i,'chainage'])
            else:
                mergesorted.loc[i,'length']=0
    endindex=len(mergesorted.index)-1 
    mergesorted.loc[endindex,'length']=0
    mergesorted=mergesorted.drop(columns=['chainage'])
    mergesorted=mergesorted[['road','id','model_type','condition','name','lat','lon', 'length']]
    mergesorted.to_csv('stap5.csv')
    
    return mergesorted

In [ ]:
m = merge_function(q[0],f2)

In [ ]:
m.to_csv('finaldf.csv',index=False)

In [ ]:
m.sort_values(by='lat')

,road,id,model_type,condition,name,lat,lon,length
1011,N1,1001011,sourcesink,NaN,"End of Road at Shapla Chattar ,Teknaf Meet wit...",20.862917,92.298083,0.0
1010,N1,1001010,link,NaN,Infor.missing,20.862972,92.298083,130.0
1009,N1,1001009,link,NaN,Km post missing,20.869277,92.298139,698.0
1008,N1,1001008,link,NaN,Teknaf 2 km,20.878222,92.297777,1000.0
1007,N1,1001007,bridge,C,BUS STAND TEKNAF BOX CULVERT,20.883070,92.298163,1.5
...,...,...,...,...,...,...,...,...
1941,N2,1001941,bridge,A,.,25.179803,92.033553,2.0
1940,N2,1001940,bridge,A,.,25.180031,92.034177,3.5
1935,N2,1001935,link,NaN,Jaflong 8 km,25.180110,92.063778,1000.0
1938,N2,1001938,bridge,A,.,25.180537,92.037383,3.5


In [ ]:
N1N2 = m.loc[((m.road == 'N1') | (m.road=='N2')) & (m.model_type == 'intersection')]
N1N2

,road,id,model_type,condition,name,lat,lon,length
15,N1,1000015,intersection,NaN,Road to Sylhet (N2),0.413749,1.579899,740.0
24,N1,1000024,intersection,NaN,Right to Syedpur (R113) left to Joydebpur N105),0.413476,1.580336,303.0
85,N1,1000085,intersection,NaN,"Z1402, road to Pennai",0.410689,1.584438,66.0
89,N1,1000089,intersection,NaN,Z1044,0.410688,1.584627,45.0
101,N1,1000101,intersection,NaN,"Z1042, road to Bancharampur",0.410373,1.586005,792.0
145,N1,1000145,intersection,NaN,Road to Brahmanbaria(N102),0.409785,1.590312,591.0
227,N1,1000227,intersection,NaN,Road to Nangolhat (Z1048),0.404038,1.593931,368.0
254,N1,1000254,intersection,NaN,Road to Karerhat (Z1031),0.402499,1.594562,84.0
257,N1,1000257,intersection,NaN,Z1031,0.402263,1.594622,521.0
267,N1,1000267,intersection,NaN,Intersection with N104 / Km post missing,0.401593,1.594906,1407.0


In [ ]:
from sklearn.neighbors import DistanceMetric
dist = DistanceMetric.get_metric('euclidean')
matrix_merged = pd.DataFrame(dist.pairwise(m[['lat','lon']].to_numpy())*6373,  columns=m.id.unique(), index=m.id.unique())
matrix_merged

NameError: ignored